In [ ]:
!pip install langchain
!pip install -qU langchain-community faiss-cpu
!pip install --upgrade --quiet  llama-cpp-python

In [ ]:
!pip install pypdf

In [1]:
from settings import *

In [2]:
from langchain_community.document_loaders import PyPDFLoader
# TODO: Can Try Other Loaders (Word Document)

loader = PyPDFLoader(PDF_FILE)
docs = loader.load()
print(f"Doc has {len(docs)} pages.")

Doc has 17 pages.


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVER, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(f"Doc is splitted {len(all_splits)} data.")

Doc is splitted 17 data.


In [4]:
# Utilizing Local LLAMA-3.1-1B to create embeding
from llama_cpp import Llama
llm = Llama(model_path=MODEL_PATH, embedding=True)
EMBED_DIM = len(llm.create_embedding(["Hello World"])['data'][0]['embedding'][0])

llama_model_loader: loaded meta data with 35 key-value pairs and 147 tensors from /home/vishal/.cache/huggingface/hub/models--bartowski--Llama-3.2-1B-Instruct-GGUF/snapshots/067b946cf014b7c697f3654f621d577a3e3afd1c/Llama-3.2-1B-Instruct-Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: 

In [5]:
# Vector Store
from typing import List
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.embeddings.embeddings import Embeddings
from utils import LlamaCppEmbeddingsFix

index = faiss.IndexFlatL2(EMBED_DIM)

embeding = LlamaCppEmbeddingsFix(
    model_path=MODEL_PATH, verbose=False,
    n_ctx=(N_DOCS_RETREIVE+2)*CHUNK_SIZE,
)

# TODO: Look at Docstore
vector_store = FAISS(
    embedding_function=embeding,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

llama_new_context_with_model: n_ctx_per_seq (7008) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [6]:
# Creating New Vector Store

vector_store.add_documents(documents=all_splits)
vector_store.save_local(VECTOR_STORE)

In [ ]:
math.ceil(len(all_splits)/BATCH_SIZE)

In [ ]:
#vector_store.search("What kind of payments does the fees management module support?", search_type="mmr",k=4)

[Document(metadata={'source': 'data/Manual Administration.pdf', 'page': 0, 'start_index': 0}, page_content='For more information, please contact\n+917970001117, +917970001018\nor\nVisit www.qlsacademy.com\nOPERATIONAL MANUAL\nExplore the world of future learning through AI-based online interactive class\nClick here to download app\nLast Update\n 17-Jan-21\nFOR INSTITUTIONAL LMS OPERATIONS'),
 Document(metadata={'source': 'data/Manual Administration.pdf', 'page': 7, 'start_index': 0}, page_content='Step-1. Click on this icon to check follow-ups History.\nStep-2. Follow-ups History.\n1\n2\n1\nStep-1. Click on Three Dots & Select Follow Up option. \nStep-2. Select Follow-up activity date.\nStep-3. Enter Activity Remark.\nStep-4. Tick Next Follow-up Check Box.\nStep-5. Select Next Action date & Next Action time.\nStep-6. Write next follow-up plans then click on Submit.\nCHECK FOLLOWUP HISTORY\nENTER NEW FOLLOWUP DETAILS\n2\n3\n5 6\n4'),
 Document(metadata={'source': 'data/Manual Administra